# Face Mask Prediction and Detection

### importing packages

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

### detect and predict mask function

In [3]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grabbing the dimensions of the frame, then constructing a blob from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
    # passing the blob through the network then performing face detection
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)
    # initializing list of faces, their corresponding locations, 
    #and the list of predictions from the face mask network
    faces = []
    locs = []
    preds = []

    # looping over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the detection
        confidence = detections[0, 0, i, 2]

        # filtering out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # computing (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensuring the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extracting the face ROI, converting it from BGR to RGB channel
            # ordering, resizing it to 224x224, and preprocessing it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # adding the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # making a predictions only if at least one face has been detected
    if len(faces) > 0:
        # making batch predictions on *all* faces at the same time rather 
        #than one-by-one predictions in the above `for` loop for faster inference    
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # returning  2-tuples of the face locations and their corresponding locations
    return (locs, preds)

### loading the serialized face detector model from disk

In [4]:
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [5]:
# loading the face mask detector model from disk
maskNet = load_model("mask_detector.model")

In [6]:
# initializing the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

[INFO] starting video stream...


### looping over the frames from the video stream

In [7]:
while True:
    # grabbing the frame from the threaded video stream and resizing it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detecting faces in the frame and determining if they are wearing a face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # looping over the detected face locations and their corresponding locations
    for (box, pred) in zip(locs, preds):
        # unpacking the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determining the class label and color for drawing
        # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # showing the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # displaying the label and bounding box rectangle on the output frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # showing the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # breaking from the loop if the `q` key is pressed. 
    if key == ord("q"):
        break

(1, 1, 105, 7)
(1, 1, 99, 7)
(1, 1, 104, 7)
(1, 1, 104, 7)
(1, 1, 103, 7)
(1, 1, 107, 7)
(1, 1, 104, 7)
(1, 1, 102, 7)
(1, 1, 104, 7)
(1, 1, 109, 7)
(1, 1, 103, 7)
(1, 1, 98, 7)
(1, 1, 105, 7)
(1, 1, 106, 7)
(1, 1, 101, 7)
(1, 1, 103, 7)
(1, 1, 104, 7)
(1, 1, 102, 7)
(1, 1, 104, 7)
(1, 1, 105, 7)
(1, 1, 107, 7)
(1, 1, 106, 7)
(1, 1, 109, 7)
(1, 1, 112, 7)
(1, 1, 96, 7)
(1, 1, 101, 7)
(1, 1, 108, 7)
(1, 1, 107, 7)
(1, 1, 111, 7)
(1, 1, 103, 7)
(1, 1, 110, 7)
(1, 1, 105, 7)
(1, 1, 105, 7)
(1, 1, 108, 7)
(1, 1, 104, 7)
(1, 1, 110, 7)
(1, 1, 111, 7)
(1, 1, 112, 7)
(1, 1, 107, 7)
(1, 1, 104, 7)
(1, 1, 107, 7)
(1, 1, 107, 7)
(1, 1, 105, 7)
(1, 1, 108, 7)
(1, 1, 103, 7)
(1, 1, 107, 7)
(1, 1, 108, 7)
(1, 1, 108, 7)
(1, 1, 108, 7)
(1, 1, 107, 7)
(1, 1, 105, 7)
(1, 1, 105, 7)
(1, 1, 114, 7)
(1, 1, 109, 7)
(1, 1, 107, 7)
(1, 1, 107, 7)
(1, 1, 104, 7)
(1, 1, 105, 7)
(1, 1, 106, 7)
(1, 1, 105, 7)
(1, 1, 106, 7)
(1, 1, 105, 7)
(1, 1, 105, 7)
(1, 1, 105, 7)
(1, 1, 108, 7)
(1, 1, 105, 7)
(1, 1, 106, 7

In [8]:
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()